In [1]:
import scrapy
import json
from scrapy.crawler import CrawlerProcess
class MySpider(scrapy.Spider):
    name = "crawler"
    start_urls = ['https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_INTRADAY&symbol=BTC&market=EUR&apikey=demo']
    
    def parse(self, response):
        print('test')
        jsonresponse = json.loads(response.body_as_unicode())
    
        yield {
            'Meta Data': jsonresponse["Meta Data"],
            'Time Series (Digital Currency Intraday)': jsonresponse["Time Series (Digital Currency Intraday)"]
            }
process = CrawlerProcess({
    'FEED_FORMAT': 'json',
    'FEED_URI': 'PythonLinks2.json',
    # Note that because we are doing API queries, the robots.txt file doesn't apply to us.
    'AUTOTHROTTLE_ENABLED': True,
    'HTTPCACHE_ENABLED': True,
    'LOG_ENABLED': False,
})

In [2]:
# Starting the crawler with our spider.
process.crawl(MySpider)
process.start()
import pandas as pd
test=pd.read_json('PythonLinks2.json', orient = 'records')
print(test['Meta Data'][0])
print(test['Meta Data'][0]['5. Market Name'])

test
{'1. Information': 'Intraday Prices and Volumes for Digital Currency', '2. Digital Currency Code': 'BTC', '3. Digital Currency Name': 'Bitcoin', '4. Market Code': 'EUR', '5. Market Name': 'Euro', '6. Interval': '5min', '7. Last Refreshed': '2018-02-08 18:30:00', '8. Time Zone': 'UTC'}
Euro
